In [1]:
!pip install bayesian-optimization

In [2]:
!pip install catboost


In [3]:
import pandas as pd                         # 데이터 분석 라이브러리
import numpy as np                          # 계산 라이브러리
import re
from tqdm import tqdm                       # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV 
from sklearn.model_selection import StratifiedKFold, cross_val_score
from bayes_opt import BayesianOptimization  # 베이지안 최적화 라이브러리  
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시

from sklearn.preprocessing import LabelEncoder

from xgboost import plot_importance
from xgboost import XGBClassifier

from catboost import CatBoostClassifier

In [4]:
from google.colab import drive
drive.mount('/gdrive/', force_remount=True)

Mounted at /gdrive/


In [5]:
!ls "/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset"

sample_submission.csv  test.csv  train.csv


In [6]:
train = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/train.csv')

In [7]:
test = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/test.csv')

In [8]:
train.head(10)

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)"
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)"
7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)"
8,0,1,0.14,0,T,Camera,"at (136.23828125, 31.26171875)"
9,0,1,0.14,0,T,Camera,"at (135.23828125, 32.26171875)"


In [9]:
unit_mineral={
    '15E0' : 50, # TrainProbe/
    '1586' : 100,   #
    '15A8' : 150,  #
    '15A0' : 150,  #
    '1581' : 125,  #
    '15C0' : 200,  #
    '15C1' : 25,  #
    '1580' : 100,  #
    '15A4' : 250,  #
    '15C3' : 275,  #
    '1585' : 50,  #
    '15A2' : 350,  #
    '15C2' : 300,  #
    '15D2' : 150,  #
    '1583' : 50,   # 
    '15A9' : 250, #
    '2720' : 400, #
    '1360' : 50, #
    '13E0' : 50, #
    '1420' : 100, #
    '13E1' : 50, #
    '13E3' : 150, #
    '1421' : 150, #
    '1407' : 150, #
    '1426' : 150, #
    '1422' : 100, #
    '1424' : 150, #
    '13E2' : 100, #
    '1423' : 400, #
    '1401' : 150,  #
    '1404' : 300,  #
    '1418' : 75,  #
    '1820' : 50,  #
    '1822' : 100,  #
    '1821' : 50,    # 
    '1823' : 100,   #
    '1829' : 75,  #
    '1BA0' : 50,  #
    '4020' : 25,  #
    '1824' : 100,   #
    '182B' : 150,  #
    '182E' : 100,  #
    '1840' : 150,  #
    '182A' : 100,   #
    '1E60' : 150,  #
    '920'  : 25   #
}


In [10]:
# 일꾼 코드 리스트 추가
worker = ['0_1360', '0_15E0', '0_1820', '1_1360','1_15E0', '1_1820']
target_worker=['0_Target_SCV','0_Target_Probe','0_Target_Drone','1_Target_SCV','1_Target_Probe','1_Target_Drone']

# 유닛 코드 리스트(프, 테, 저) 추가 

# 프로토스 유닛 코드
              #  Adept  Oracle  Phoenix  Stalker  WarpPrism  Observer  Zealot  VoidRay  Immortal  Sentry  Carrier  Colossus  Disruptor  HighTemplar  Tempest  MothershipCore
unit_code_P_0 = ['0_1586', '0_15A8', '0_15A0', '0_1581', '0_15C0', '0_15C1', '0_1580', '0_15A4', '0_15C3', '0_1585', '0_15A2', '0_15C2', '0_15D2', '0_1583', '0_15A9', '0_2720']
unit_code_P_1 = ['1_1586', '1_15A8', '1_15A0', '1_1581', '1_15C0', '1_15C1', '1_1580', '1_15A4', '1_15C3', '1_1585', '1_15A2', '1_15C2', '1_15D2', '1_1583', '1_15A9', '1_2720']          

# 테란 유닛 코드 
              #  Marine  Medivac  Reaper  Marauder  Banshee  Cyclone  Liberator  Raven  Viking  Ghost  Battlecruiser  Crucio  Thor  Mine
unit_code_T_0 = ['0_13E0', '0_1420', '0_13E1', '0_13E3', '0_1421', '0_1407', '0_1426', '0_1422', '0_1424', '0_13E2', '0_1423', '0_1401', '0_1404', '0_1418']
unit_code_T_1 = ['1_13E0', '1_1420', '1_13E1', '1_13E3', '1_1421', '1_1407', '1_1426', '1_1422', '1_1424', '1_13E2', '1_1423', '1_1401', '1_1404', '1_1418']

# 저그 유닛 코드
              #  Overlord  Zergling  Hydralisk  Roach  ToOverseer  ToRavager  Mutalisk  Corruptor  SwarmHost  ToBroodLord  Infestor  Baneling 
unit_code_Z_0 = ['0_1822', '0_1821', '0_1823', '0_1829', '0_1BA0', '0_4020', '0_1824', '0_182B', '0_182E', '0_1840', '0_182A', '0_1E60', '0_920']
unit_code_Z_1 = ['1_1822', '1_1821', '1_1823', '1_1829', '1_1BA0', '1_4020', '1_1824', '1_182B', '1_182E', '1_1840', '1_182A', '1_1E60', '1_920']

total_unit_code = unit_code_P_0 + unit_code_P_1 + unit_code_T_0 + unit_code_T_1 + unit_code_Z_0 + unit_code_Z_1

In [11]:
def need_swap(row):
    """
    player swap이 필요한지 확인 (저테프 를 프테저로)
    T vs P -> P vs T
    Z vs P -> P vs Z
    Z vs T -> T vs Z
    """

    if row['species_0'] == 'T' and row['species_1'] == 'P':
        return 1
    elif row['species_0'] == 'Z' and row['species_1'] == 'P':
        return 1
    elif row['species_0'] == 'Z' and row['species_1'] == 'T':
        return 1
    else:
        return 0

In [12]:
def winner_swap(org_winner):
  if org_winner == 0:
    return 1
  elif org_winner == 1:
    return 0
  else:
    raise ValueError('Incorrect Number')

In [13]:
def swap(train, unit_code_unique, target_unique, selection_unique, answer=False):
  events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']

  swaped_index = train.loc[train.swap == 1].index.values

  temp_species = train.loc[train.swap == 1, 'species_0']
  train.loc[train.swap == 1, 'species_0'] = train.loc[train.swap == 1, 'species_1']
  train.loc[train.swap == 1, 'species_1'] = temp_species

  temp_event_count = train.loc[train.swap == 1, 'event_count_0']
  train.loc[train.swap == 1, 'event_count_0'] = train.loc[train.swap == 1, 'event_count_1']
  train.loc[train.swap == 1, 'event_count_1'] = temp_event_count

  temp_event_per_sec = train.loc[train.swap == 1, 'event_per_sec_0']
  train.loc[train.swap == 1, 'event_per_sec_0'] = train.loc[train.swap == 1, 'event_per_sec_1']
  train.loc[train.swap == 1, 'event_per_sec_1'] = temp_event_per_sec

  temp_worker = train.loc[train.swap == 1, 'worker_0']
  train.loc[train.swap == 1, 'worker_0'] = train.loc[train.swap == 1, 'worker_1']
  train.loc[train.swap == 1, 'worker_1'] = temp_worker

  temp_target_worker = train.loc[train.swap == 1, 'target_worker_0']
  train.loc[train.swap == 1, 'target_worker_0'] = train.loc[train.swap == 1, 'target_worker_1']
  train.loc[train.swap == 1, 'target_worker_1'] = temp_target_worker

  temp_unit_count = train.loc[train.swap == 1, 'unit_count_0']
  train.loc[train.swap == 1, 'unit_count_0'] = train.loc[train.swap == 1, 'unit_count_1']
  train.loc[train.swap == 1, 'unit_count_1'] = temp_unit_count

  temp_unit_mineral = train.loc[train.swap == 1, 'unit_mineral_0']
  train.loc[train.swap == 1, 'unit_mineral_0'] = train.loc[train.swap == 1, 'unit_mineral_1']
  train.loc[train.swap == 1, 'unit_mineral_1'] = temp_unit_mineral

  for event in events:
    # 이벤트들
    temp_event = train.loc[train.swap == 1, event + '_count_0'] 
    train.loc[train.swap == 1, event + '_count_0']  = train.loc[train.swap == 1, event + '_count_1'] 
    train.loc[train.swap == 1, event + '_count_1']  = temp_event
    # diff 는 -1
    train.loc[train.swap == 1, event + '_diff'] = (train.loc[train.swap == 1, event + '_diff']) * -1

  for unit_code in unit_code_unique:
    try:
      temp_unit_code = train.loc[train.swap == 1, '0_' + unit_code]    
      train.loc[train.swap == 1, '0_' + unit_code]   = train.loc[train.swap == 1, '1_' + unit_code]  
      train.loc[train.swap == 1, '1_' + unit_code]  = temp_unit_code

      train.loc[train.swap == 1, unit_code + '_diff'] = (train.loc[train.swap == 1, unit_code + '_diff']) * -1
    except KeyError as e:
      print('key error!!! ======', e)

  for target in target_unique:
    try:
      temp_target = train.loc[train.swap == 1, '0_Target_' + target]     
      train.loc[train.swap == 1, '0_Target_' + target] = train.loc[train.swap == 1, '1_Target_' + target] 
      train.loc[train.swap == 1, '1_Target_' + target]  = temp_target
    except KeyError as e:
      print('key error!!! ======', e)

  for selection in selection_unique:
    try:
      temp_selection = train.loc[train.swap == 1, '0_' + selection]     
      train.loc[train.swap == 1, '0_' + selection] = train.loc[train.swap == 1, '1_' + selection] 
      train.loc[train.swap == 1, '1_' + selection]  = temp_selection
    except KeyError as e:
      print('key error!!! ======', e)

  train.loc[train.swap == 1, 'event_per_sec_diff'] = (train.loc[train.swap == 1, 'event_per_sec_diff']) * -1
  train.loc[train.swap == 1, 'worker_diff'] = (train.loc[train.swap == 1, 'worker_diff']) * -1
  
  # winner 반전
  if answer:
    train.loc[train.swap == 1, 'winner'] = train.loc[train.swap == 1, 'winner'].apply(lambda x: winner_swap(x))

  train.drop(labels='swap',axis=1, inplace=True)
  return swaped_index

In [14]:
train['species'].unique()

array(['T', 'Z', 'P'], dtype=object)

In [15]:
# Label Encoding

species_encoder = LabelEncoder()
species_encoder.fit(train['species'].unique())

#Encoding 결과 확인
# 0:프, 1:테, 2:저
species_encoder.classes_

array(['P', 'T', 'Z'], dtype=object)

In [16]:
def data_preparation(train, answer=False):
    df_train = pd.DataFrame(columns=['game_id'])
    df_train.game_id = train.game_id.unique()
    
    df_train['time'] = np.array(train[train.shift(-1).game_id != train.game_id].time)
    df_train['time'] = (df_train.time*100//100*60 + df_train.time*100%100).astype(int) #시간을 초단위로 변경

    df = train[train.player==0]
    df_train['species_0']=np.array(df[df.shift(-1).game_id != df.game_id].species)
    df = train[train.player==1]
    df_train['species_1']=np.array(df[df.shift(-1).game_id != df.game_id].species)

    df_train['swap'] = df_train.apply(lambda row: need_swap(row), axis=1)

    df = train[train.player==0]
    df_train['event_count_0'] = np.array(df.game_id.value_counts()[df.game_id.unique()])
    df = train[train.player==1]
    df_train['event_count_1'] = np.array(df.game_id.value_counts()[df.game_id.unique()])

    df_train['event_per_sec_0'], df_train['event_per_sec_1'] = df_train['event_count_0'] /df_train.time, df_train['event_count_1'] /df_train.time  # 이벤트당 걸린 시간

    events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']
    for event in events:
        df = train[(train.player==0)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_0'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')
    
        df = train[(train.player==1)&(train.event==event)]
        df = pd.DataFrame(df.game_id.value_counts()[df.game_id.unique()]).rename({'game_id':event+'_count_1'}, axis = 1)
        df['game_id']= np.array(df.index)
        df_train = pd.merge(df_train, df, on='game_id', how='left')
    df_train = df_train.fillna(0)

    df_train.index = df_train['game_id']
    df_train = df_train.drop(labels='game_id',axis=1)

    df_total_unit_code = []
    contents = pd.DataFrame(train.event_contents[(train.event == 'Ability')].map(lambda x: x[x.find('(')+1:x.find(')')]))  # event_contents의 16진수 코드만 추출
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    unit_code_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        if i in worker:
            df_total_unit_code.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        if i in total_unit_code:
            df_total_unit_code.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)
    df_train['worker_0'],df_train['worker_1'] = df_train['0_1360'] + df_train['0_1820'] + df_train['0_15E0'],  df_train['1_1360'] + df_train['1_1820'] + df_train['1_15E0']
    
    df_total_unit_target =[]
    contents = pd.DataFrame(train.event_contents[(train.event == 'Right Click') & (train.event_contents.map(lambda x: str(x)[:6]) == 'Target')].map(lambda x: x[x.find(':')+2:x.find(' [')]))  # event_contents의 Target만 추출
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    target_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_Target_','1_Target_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_Target_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        if i in target_worker:
            df_total_unit_target.append(i)
            df_train[i] = contents_X[i]
            df_train[i] = df_train[i].fillna(0).astype(int)
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)
    
    df_train['target_worker_0'],df_train['target_worker_1'] = df_train['0_Target_SCV'] + df_train['0_Target_Probe'] + df_train['0_Target_Drone'],  df_train['1_Target_SCV'] + df_train['1_Target_Probe'] + df_train['1_Target_Drone']

    # event == Selection일 때 event_contents 더미 변수 생성, 카운트
    contents = train[train.event == 'Selection'].event_contents.map(lambda x: re.sub('\s\[.....\]', '', re.sub('\s\[......\]', '', re.sub('\s\[.......\]', '', x))).
                                                            replace('[', '').replace(']', '').replace(' ', '').replace('\'', ''))
    contents = contents.str.split(',')
    max_num = max(contents.map(lambda x: len(x)))
    t = [0 for x in range(max_num)]
    for i in range(max_num):
        t[i] = pd.DataFrame(contents[contents.map(lambda x: len(x) > i)].map(lambda x: x[i]))

    contents = pd.concat([t[i] for i in range(max_num)])
    contents['game_id'], contents['player'], contents['count'] = train.game_id, train.player, 1
    selection_unique = contents.event_contents.unique().copy()
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in contents_X.columns:
        df_train[i] = contents_X[i]
        df_train[i] = df_train[i].fillna(0).astype(int)

    unit_count_0 = 0
    unit_count_1 = 0
    for j in unit_code_P_0 + unit_code_T_0 + unit_code_Z_0:
        try:
            unit_count_0 += df_train[j]
        except KeyError:    # 데이터에 해당 유닛 코드가 없을 때
            unit_count_0 += 0

    for j in unit_code_P_1 + unit_code_T_1 + unit_code_Z_1:
        try:
            unit_count_1 += df_train[j]
        except KeyError:
            unit_count_1 += 0

    df_train['unit_count_0'], df_train['unit_count_1'] = unit_count_0, unit_count_1

    unit_mineral_0 = 0
    unit_mineral_1 = 0
    for i in unit_code_P_0 + unit_code_T_0 + unit_code_Z_0:
        try:
            # print('unit code  :  ', i)
            # print('unit code count :  ', df_train[i])
            # print('unit code count mineral:  ', df_train[i] * unit_mineral[i.split('_')[1]])
            # print('=====================================')
            unit_mineral_0 += df_train[i] * unit_mineral[i.split('_')[1]]
        except KeyError:
            unit_mineral_0 +=0

    for i in unit_code_P_1 + unit_code_T_1 + unit_code_Z_1:
        try:
            # print('unit code  :  ', i)
            # print('unit code count :  ', df_train[i])
            # print('unit code count mineral:  ', df_train[i] * unit_mineral[i.split('_')[1]])
            # print('=====================================')
            unit_mineral_1 += df_train[i] * unit_mineral[i.split('_')[1]]
        except KeyError:
            unit_mineral_1 +=0

    df_train['unit_mineral_0'] = unit_mineral_0
    df_train['unit_mineral_1'] = unit_mineral_1


    df_train.drop(labels=worker, axis=1, inplace=True)
    df_train.drop(labels=target_worker, axis=1, inplace=True)
    # df_train.drop(labels=df_total_unit_code , axis=1, inplace=True)
    # df_train.drop(labels=df_total_unit_target , axis=1, inplace=True)

    for event in events:
        df_train[event + '_diff'] = df_train[event+'_count_0'] - df_train[event+'_count_1']

    df_train['event_per_sec_diff'] = df_train['event_per_sec_0'] - df_train['event_per_sec_1']

    df_train['worker_diff'] = df_train['worker_0'] - df_train['worker_1']

    for unit_code in unit_code_unique:
        try:
            df_train[unit_code + '_diff'] = df_train['0_' + unit_code] - df_train['1_' + unit_code]
        except KeyError as e:
          pass

    if answer:
        df_train['winner'] = np.array(train[train.shift(-1).game_id != train.game_id].winner)

    df_train['species_0'] = species_encoder.transform(df_train['species_0'])
    df_train['species_1'] = species_encoder.transform(df_train['species_1'])  

    swaped_index = swap(df_train, unit_code_unique, target_unique, selection_unique, answer=answer)

    if answer:
        x_data = df_train.iloc[:, :-1]
    else:
        x_data = df_train.iloc[:, :]
    y_data = pd.Series()

    x_data = x_data[sorted(x_data.columns)]

    if answer:
        y_data = df_train['winner']
    return x_data, y_data, swaped_index

In [17]:
x_train, y_train, swaped_index = data_preparation(train, answer=True)

key error!!! ====== '0_1360'
key error!!! ====== '0_1820'
key error!!! ====== '0_15E0'
key error!!! ====== '0_Target_Probe'
key error!!! ====== '0_Target_SCV'
key error!!! ====== '0_Target_Drone'


In [18]:
x_test, _, swaped_index = data_preparation(test, answer=False)

key error!!! ====== '0_15E0'
key error!!! ====== '0_1820'
key error!!! ====== '0_1360'
key error!!! ====== '0_Target_Probe'
key error!!! ====== '0_Target_SCV'
key error!!! ====== '0_Target_Drone'


In [19]:
del train
del test

In [20]:
swaped_index[:10]

array([38873, 38875, 38882, 38885, 38890, 38892, 38898, 38900, 38901,
       38906])

In [21]:
x_train.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,...,F01_diff,F40_diff,F41_diff,F60_diff,F61_diff,F80_diff,F81_diff,FC0_diff,FC1_diff,FE0_diff,FE1_diff,GetControlGroup_count_0,GetControlGroup_count_1,GetControlGroup_diff,Right Click_count_0,Right Click_count_1,Right Click_diff,Selection_count_0,Selection_count_1,Selection_diff,SetControlGroup_count_0,SetControlGroup_count_1,SetControlGroup_diff,event_count_0,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,target_worker_0,target_worker_1,time,unit_count_0,unit_count_1,unit_mineral_0,unit_mineral_1,worker_0,worker_1,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,24.0,3.0,21.0,35.0,28.0,7.0,50.0,57.0,-7.0,3.0,1.0,2.0,592,548,1.333333,1.234234,0.099099,1,1,0,0,444,5,7,325,550,9,6,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1,0,192.0,43.0,149.0,303.0,79.0,224.0,87.0,148.0,-61.0,5.0,17.0,-12.0,1165,655,2.600446,1.462054,1.138393,0,2,1,0,448,8,7,1000,850,22,23,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,3.0,322.0,-319.0,86.0,111.0,-25.0,36.0,91.0,-55.0,2.0,5.0,-3.0,434,815,1.631579,3.063910,-1.432331,0,0,2,0,266,3,4,300,775,7,8,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,138.0,8.0,130.0,409.0,159.0,250.0,91.0,85.0,6.0,1.0,0.0,1.0,1236,1155,2.063439,1.928214,0.135225,0,1,2,2,599,5,7,900,450,16,9,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,47.0,41.0,6.0,68.0,126.0,-58.0,103.0,95.0,8.0,5.0,7.0,-2.0,477,393,1.616949,1.332203,0.284746,2,2,0,1,295,4,9,250,600,6,6,0


In [22]:
x_train.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_0  worker_1  worker_diff
game_id                      ...                                 
0         5       0       1  ...         9         6            3
7        13       0       0  ...        22        23           -1
14        0       0       0  ...         7         8           -1
16        4       0       0  ...        16         9            7
18        6       0       0  ...         6         6            0
...      ..     ...     ...  ...       ...       ...          ...
38865     0       0       0  ...         3         1            2
38866     0       0       0  ...         2         2            0
38867     2       0       0  ...         6        11           -5
38868     0       0       0  ...        11         7            4
38871    18       0       2  ...        11        26          -15

[19436 rows x 1862 columns]>

In [23]:
y_train.head(10)

game_id
0     1
7     1
14    1
16    1
18    1
20    1
21    0
23    1
24    0
25    0
Name: winner, dtype: int64

In [24]:
print(x_train.shape)
print(y_train.shape)

(19436, 1862)
(19436,)


In [25]:
x_test.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,...,F01_diff,F40_diff,F41_diff,F60_diff,F61_diff,F80_diff,F81_diff,FC0_diff,FC1_diff,FE0_diff,FE1_diff,GetControlGroup_count_0,GetControlGroup_count_1,GetControlGroup_diff,Right Click_count_0,Right Click_count_1,Right Click_diff,Selection_count_0,Selection_count_1,Selection_diff,SetControlGroup_count_0,SetControlGroup_count_1,SetControlGroup_diff,event_count_0,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,target_worker_0,target_worker_1,time,unit_count_0,unit_count_1,unit_mineral_0,unit_mineral_1,worker_0,worker_1,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
38872,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2,0,9.0,43.0,-34.0,241.0,122.0,119.0,52.0,41.0,11.0,5.0,0.0,5.0,571,715,1.340376,1.678404,-0.338028,0,0,1,0,426,2,4,525,375,11,12,-1
38873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,-1,0,0,0,0,0,0,0,0,0,122.0,18.0,104.0,68.0,51.0,17.0,19.0,114.0,-95.0,5.0,0.0,5.0,389,421,1.059946,1.147139,-0.087193,0,2,0,0,367,4,11,675,1150,13,7,6
38874,1,0,0,3,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,-1,0,0,0,0,0,63.0,426.0,-363.0,79.0,108.0,-29.0,42.0,36.0,6.0,7.0,5.0,2.0,469,824,1.669039,2.932384,-1.263345,1,1,0,1,281,3,1,250,150,9,5,4
38875,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,-2,0,0,0,0,0,24.0,255.0,-231.0,322.0,218.0,104.0,99.0,123.0,-24.0,2.0,9.0,-7.0,863,1223,1.805439,2.558577,-0.753138,0,1,1,5,478,3,10,400,775,19,21,-2
38876,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,387.0,1.0,386.0,30.0,7.0,23.0,25.0,35.0,-10.0,2.0,1.0,1.0,582,213,3.423529,1.252941,2.170588,2,2,0,0,170,0,2,0,150,4,2,2


In [26]:
x_test.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_0  worker_1  worker_diff
game_id                      ...                                 
38872     4       0       0  ...        11        12           -1
38873     1       0       0  ...        13         7            6
38874     1       0       0  ...         9         5            4
38875    11       0       0  ...        19        21           -2
38876     0       0       0  ...         4         2            2
...      ..     ...     ...  ...       ...       ...          ...
55654     4       0       1  ...        19         7           12
55655    11       0       1  ...        10        11           -1
55656     6       0       0  ...         6         8           -2
55657     0       0       0  ...         4         2            2
55658     4       0       0  ...         4         6           -2

[16787 rows x 1852 columns]>

In [27]:
useless = []
for c in x_train.columns:
    try:
        if x_train[c].nunique()<2:
            useless.append(c)
            print("useless", c)
    except:
        print("exception:", c)

useless 0_11E0
useless 0_152C
useless 0_152F
useless 0_1780
useless 0_17C0
useless 0_1826
useless 0_182A
useless 0_182B
useless 0_1840
useless 0_1860
useless 0_1880
useless 0_18A0
useless 0_18C0
useless 0_1980
useless 0_1A63
useless 0_1AE0
useless 0_1B00
useless 0_1B23
useless 0_1B60
useless 0_1BE3
useless 0_2061
useless 0_2101
useless 0_2240
useless 0_2D80
useless 0_2DA0
useless 0_3161
useless 0_3FC0
useless 0_4041
useless 0_42A3
useless 0_5641
useless 0_960
useless 0_A80
useless 0_Baneling2
useless 0_BanelingCocoon2
useless 0_BanelingNest2
useless 0_BroodLord
useless 0_ChangelingMarineShield
useless 0_Corruptor
useless 0_DA0
useless 0_Disruptor
useless 0_EA2
useless 0_Egg2
useless 0_ExtendingBridgeNWWide8
useless 0_Infestor
useless 0_InfestorBurrowed
useless 0_Lurker
useless 0_SCV2
useless 0_Starport2
useless 0_Target_BanelingBurrowed
useless 0_Target_Beacon_Zerg
useless 0_Target_Disruptor
useless 0_Target_Infestor
useless 0_Ultralisk
useless 0_UltraliskCavern
useless 1_10C0
useless 

In [28]:
x_train.drop(labels=useless, axis=1, inplace=True)

In [29]:
x_train.head()

,0_,0_1000,0_1020,0_1021,0_1022,0_1023,0_1024,0_1025,0_1026,0_1028,0_1029,0_102A,0_102B,0_102D,0_102F,0_103E,0_1060,0_1080,0_1081,0_10C0,0_10E0,0_1100,0_1120,0_1121,0_1140,0_1142,0_1143,0_1160,0_1180,0_11A0,0_11C0,0_11E1,0_11E3,0_1201,0_1203,0_1204,0_1220,0_1240,0_1260,0_1261,...,F01_diff,F40_diff,F41_diff,F60_diff,F61_diff,F80_diff,F81_diff,FC0_diff,FC1_diff,FE0_diff,FE1_diff,GetControlGroup_count_0,GetControlGroup_count_1,GetControlGroup_diff,Right Click_count_0,Right Click_count_1,Right Click_diff,Selection_count_0,Selection_count_1,Selection_diff,SetControlGroup_count_0,SetControlGroup_count_1,SetControlGroup_diff,event_count_0,event_count_1,event_per_sec_0,event_per_sec_1,event_per_sec_diff,species_0,species_1,target_worker_0,target_worker_1,time,unit_count_0,unit_count_1,unit_mineral_0,unit_mineral_1,worker_0,worker_1,worker_diff
game_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,5,0,1,4,2,2,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0,0,24.0,3.0,21.0,35.0,28.0,7.0,50.0,57.0,-7.0,3.0,1.0,2.0,592,548,1.333333,1.234234,0.099099,1,1,0,0,444,5,7,325,550,9,6,3
7,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1,0,192.0,43.0,149.0,303.0,79.0,224.0,87.0,148.0,-61.0,5.0,17.0,-12.0,1165,655,2.600446,1.462054,1.138393,0,2,1,0,448,8,7,1000,850,22,23,-1
14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,3.0,322.0,-319.0,86.0,111.0,-25.0,36.0,91.0,-55.0,2.0,5.0,-3.0,434,815,1.631579,3.063910,-1.432331,0,0,2,0,266,3,4,300,775,7,8,-1
16,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,138.0,8.0,130.0,409.0,159.0,250.0,91.0,85.0,6.0,1.0,0.0,1.0,1236,1155,2.063439,1.928214,0.135225,0,1,2,2,599,5,7,900,450,16,9,7
18,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,47.0,41.0,6.0,68.0,126.0,-58.0,103.0,95.0,8.0,5.0,7.0,-2.0,477,393,1.616949,1.332203,0.284746,2,2,0,1,295,4,9,250,600,6,6,0


In [30]:
# x_train, x_test에만 있는 컬럼 삭제
x_train.drop(set(x_train.columns) - set(x_test.columns), axis=1, inplace=True)
x_test.drop(set(x_test.columns) - set(x_train.columns), axis=1, inplace=True)

In [31]:
x_train.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_0  worker_1  worker_diff
game_id                      ...                                 
0         5       0       1  ...         9         6            3
7        13       0       0  ...        22        23           -1
14        0       0       0  ...         7         8           -1
16        4       0       0  ...        16         9            7
18        6       0       0  ...         6         6            0
...      ..     ...     ...  ...       ...       ...          ...
38865     0       0       0  ...         3         1            2
38866     0       0       0  ...         2         2            0
38867     2       0       0  ...         6        11           -5
38868     0       0       0  ...        11         7            4
38871    18       0       2  ...        11        26          -15

[19436 rows x 1737 columns]>

In [32]:
x_test.info

<bound method DataFrame.info of          0_  0_1000  0_1020  ...  worker_0  worker_1  worker_diff
game_id                      ...                                 
38872     4       0       0  ...        11        12           -1
38873     1       0       0  ...        13         7            6
38874     1       0       0  ...         9         5            4
38875    11       0       0  ...        19        21           -2
38876     0       0       0  ...         4         2            2
...      ..     ...     ...  ...       ...       ...          ...
55654     4       0       1  ...        19         7           12
55655    11       0       1  ...        10        11           -1
55656     6       0       0  ...         6         8           -2
55657     0       0       0  ...         4         2            2
55658     4       0       0  ...         4         6           -2

[16787 rows x 1737 columns]>

In [33]:
def cat_cv(learning_rate, n_estimators, subsample, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = CatBoostClassifier(
            # eval_metric = 'AUC',  
            # task_type = 'GPU',   
            # grow_policy = 'Depthwise', 
            # num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            # colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            # reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [34]:
# 모델과 관련없는 변수 고정
func_fixed1 = partial(cat_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
catBO = BayesianOptimization(
    func_fixed1, 
    {
        # 'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        # 'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        # 'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
catBO.maximize(init_points=5, n_iter=10) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
358:	learn: 0.5758318	total: 24s	remaining: 6.87s
359:	learn: 0.5755891	total: 24s	remaining: 6.8s
360:	learn: 0.5753007	total: 24.1s	remaining: 6.74s
361:	learn: 0.5751127	total: 24.2s	remaining: 6.67s
362:	learn: 0.5748923	total: 24.2s	remaining: 6.61s
363:	learn: 0.5747285	total: 24.3s	remaining: 6.54s
364:	learn: 0.5745447	total: 24.3s	remaining: 6.47s
365:	learn: 0.5743809	total: 24.4s	remaining: 6.4s
366:	learn: 0.5742330	total: 24.5s	remaining: 6.33s
367:	learn: 0.5739916	total: 24.5s	remaining: 6.26s
368:	learn: 0.5737145	total: 24.6s	remaining: 6.19s
369:	learn: 0.5734583	total: 24.6s	remaining: 6.13s
370:	learn: 0.5732344	total: 24.7s	remaining: 6.06s
371:	learn: 0.5726725	total: 24.8s	remaining: 5.99s
372:	learn: 0.5724465	total: 24.8s	remaining: 5.92s
373:	learn: 0.5722755	total: 24.9s	remaining: 5.85s
374:	learn: 0.5720726	total: 24.9s	remaining: 5.78s
375:	learn: 0.5719026	total: 25s	remaining: 5.71s
376:	learn: 0.5716526	total: 25s	rem

In [35]:
params1 = catBO.max['params']
models1 = cat_cv(
    # params1['num_leaves'], 
    params1['learning_rate'], 
    params1['n_estimators'], 
    params1['subsample'], 
    # params1['colsample_bytree'], 
    # params1['reg_alpha'], 
    params1['reg_lambda'], 
    x_data=x_train, y_data=y_train, n_splits=5, output='model')

0:	learn: 0.6896366	total: 48.8ms	remaining: 29.5s
1:	learn: 0.6867426	total: 98.1ms	remaining: 29.6s
2:	learn: 0.6844836	total: 153ms	remaining: 30.7s
3:	learn: 0.6823586	total: 200ms	remaining: 30.2s
4:	learn: 0.6800431	total: 259ms	remaining: 31.1s
5:	learn: 0.6779109	total: 316ms	remaining: 31.6s
6:	learn: 0.6758626	total: 366ms	remaining: 31.3s
7:	learn: 0.6741254	total: 417ms	remaining: 31.1s
8:	learn: 0.6727275	total: 468ms	remaining: 31s
9:	learn: 0.6712291	total: 523ms	remaining: 31.1s
10:	learn: 0.6699190	total: 575ms	remaining: 31.1s
11:	learn: 0.6687368	total: 624ms	remaining: 30.9s
12:	learn: 0.6674930	total: 679ms	remaining: 31s
13:	learn: 0.6663935	total: 735ms	remaining: 31.1s
14:	learn: 0.6654066	total: 791ms	remaining: 31.2s
15:	learn: 0.6641704	total: 842ms	remaining: 31s
16:	learn: 0.6631301	total: 893ms	remaining: 31s
17:	learn: 0.6624617	total: 942ms	remaining: 30.8s
18:	learn: 0.6611324	total: 992ms	remaining: 30.6s
19:	learn: 0.6599106	total: 1.04s	remaining: 30

In [36]:
preds1 = []
for model in models1:
    pred = model.predict_proba(x_test)[:, 1]
    preds1.append(pred)
pred1 = np.mean(preds1, axis=0)

In [37]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    models = []
    for train_index, valid_index in skf.split(x_data, y_data):
        x_train, y_train = x_data.iloc[train_index], y_data.iloc[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data.iloc[valid_index]
        
        model = lgb.LGBMClassifier(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict_proba(x_valid)[:, 1]
        true = y_valid
        score += roc_auc_score(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [38]:
# 모델과 관련없는 변수 고정
func_fixed2 = partial(lgb_cv, x_data=x_train, y_data=y_train, n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed2, 
    {
        'num_leaves': (16, 1024),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=10) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!
# LightGBM Classifier: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------
|  1        |  0.7264   |  0.0708   |  0.08152  |  790.0    |  304.6    |  1.931    |  48.95    |  0.4062   |
|  2        |  0.7186   |  0.7578   |  0.009006 |  328.4    |  639.9    |  4.599    |  10.92    |  0.6635   |
|  3        |  0.7255   |  0.6787   |  0.09504  |  299.5    |  640.8    |  3.833    |  20.02    |  0.9427   |
|  4        |  0.7203   |  0.9299   |  0.09484  |  394.5    |  361.0    |  6.648    |  2.116    |  0.2322   |
|  5        |  0.7322   |  0.4301   |  0.007886 |  788.7    |  876.7    |  1.504    |  5.057    |  0.2709   |
|  6        |  0.6844   |  0.1846   |  0.08544  |  23.89    |  18.39    |  1.073    |  47.99    |  0.7384   |
|  7        |  0.6855   |  0.9608   |  0.02043  |  27.04    |  1.023e+0 |  3.621    |  6.104    |  0.5903   |
|  8      

In [39]:
params2 = lgbBO.max['params']
models2 = lgb_cv(
    params2['num_leaves'], 
    params2['learning_rate'], 
    params2['n_estimators'], 
    params2['subsample'], 
    params2['colsample_bytree'], 
    params2['reg_alpha'], 
    params2['reg_lambda'], 
    x_data=x_train, y_data=y_train, n_splits=5, output='model')

In [40]:
preds2 = []
for model in models2:
    pred = model.predict_proba(x_test)[:, 1]
    preds2.append(pred)
pred2 = np.mean(preds2, axis=0)

In [41]:
submission1 = pd.read_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/dataset/sample_submission.csv', index_col=0)
submission2 = submission1.copy()

In [44]:
submission1['winner'] = submission1['winner'] + pred1 # pred는 pred1 과 pred2의 비율

In [45]:
submission1.head()

,winner
game_id,
38872,0.522795
38873,0.421300
38874,0.509124
38875,0.762980
38876,0.554765


In [46]:
submission2['winner'] = submission2['winner'] + pred2

In [47]:
submission2.head()

,winner
game_id,
38872,0.496324
38873,0.427436
38874,0.465449
38875,0.867533
38876,0.541024


In [48]:
for i in swaped_index:
    submission1.loc[i, :] = 1 - submission1.loc[i, :]
    submission2.loc[i, :] = 1 - submission2.loc[i, :]

In [49]:
submission1.head()

,winner
game_id,
38872,0.522795
38873,0.578700
38874,0.509124
38875,0.237020
38876,0.554765


In [50]:
submission2.head()

,winner
game_id,
38872,0.496324
38873,0.572564
38874,0.465449
38875,0.132467
38876,0.541024


In [51]:
submission1['winner'] = submission1['winner']*0.6 + submission2['winner']*0.4

In [52]:
submission1.head()

,winner
game_id,
38872,0.512207
38873,0.576246
38874,0.491654
38875,0.195199
38876,0.549269


In [53]:
submission1.to_csv('/gdrive/My Drive/Colab Notebooks/kaggleCompete/star2/results/submission_master2.csv')